# 이번 장에서 사용할 라이브러리 설치

- transformers: 허깅페이스에서 제공하는 트랜스포머용 라이브러리
- datasets: 허깅페이스에서 제공하는 다양한 공개 데이터셋을 가져올 수 있음
- accelerate: 하드웨어 가속기를 효율적으로 사용할 수 있게 함
- peft: parameter-efficient fine-tuning
- bitsandbytes: 양자화 기법 제공

- -qqq: 강한 억제 옵션으로, 오류 메시지를 제외한 거의 모든 출력을 억제; 설치 중에 필요한 절대 최소한의 메시지만 출력

In [ ]:
!pip install transformers==4.40.1 datasets==2.19.0 accelerate==0.30.0 peft==0.10.0 bitsandbytes==0.43.1 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requir

# 메모리 사용량 측정을 위한 함수 구현

In [ ]:
import torch

def print_gpu_utilization(step=""):
    if torch.cuda.is_available():
      """GPU 메모리 사용량 추적 및 출력 함수."""
      used_memory = torch.cuda.memory_allocated() / 1024**3 # 비트 단위를 GB 단위로 변경
      print(f"[{step}] GPU 메모리 사용량: {used_memory:.3f} GB")
    else:
        print("런타임 유형을 GPU로 변경하세요")

print_gpu_utilization()
# 출력 결과
# GPU 메모리 사용량: 0.000 GB

[] GPU 메모리 사용량: 0.000 GB


# 모델을 불러오고 GPU 메모리와 데이터 타입 확인

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model_and_tokenizer(model_id, peft=None): # peft에 아무 것도 정의하지 않으면 기본 상태
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        # torch_dtype에 아무 것도 안 넣으면 기본적인 데이터 타입이 들어감. device_map을 0으로 설정 -> 어차피 랩탑은 GPU 하나니까
    print_gpu_utilization("load_model_and_tokenizer")
    return model, tokenizer

model_id = "EleutherAI/polyglot-ko-1.3b"
model, tokenizer = load_model_and_tokenizer(model_id) # GPU 메모리 사용량: 2.599 GB
print("모델 파라미터 데이터 타입: ", model.dtype) # torch.float16

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00003.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/748M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

[load_model_and_tokenizer] GPU 메모리 사용량: 2.599 GB
모델 파라미터 데이터 타입:  torch.float16


# 그레이디언트와 옵티마이저 상태의 메모리 사용량을 계산하는 함수

In [ ]:
from transformers import AdamW
from torch.utils.data import DataLoader

# .nelement(): gradient tensor의 요소 개수 반환
# .element_size(): 각 요소가 차지하는 바이트 크기 반환

def estimate_memory_of_gradients(model):
    total_memory = 0
    for param in model.parameters():
        if param.grad is not None:
            total_memory += param.grad.nelement() * param.grad.element_size()
    return total_memory

def estimate_memory_of_optimizer(optimizer):
    total_memory = 0
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                total_memory += v.nelement() * v.element_size()
    return total_memory

# 모델의 학습 과정에서 메모리 사용량을 확인하는 train_model 정의

In [ ]:
# 모델과 데이터셋, 학습에 필요한 매개변수를 받아 학습을 수행하는데,
def train_model(model, dataset, training_args):

    # gradient checkpointing 기능이 활성화돼있다면, 이번 학습에 사용하도록 함.
    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # 데이터 로더와 옵티마이저 설정
    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())

    # 모델 학습 모드로 전환
    model.train()

    # 배치 단위로 반복 학습
    gpu_utilization_printed = False
    for step, batch in enumerate(train_dataloader, start=1):

        # 배치에 있는 데이터 GPU 디바이스로 전송
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # 순전파 결과에서 loss를 줄이고,
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / training_args.gradient_accumulation_steps

        # 역전파 수행
        loss.backward()

        # 나머지가 0이라는 것은 옵티마이저를 업데이트 할 때가 됐다는 소리.
        # 근데 training_args.gradient_accumulation_steps 가 1(default)이면 항상 수행되는 코드.
        if step % training_args.gradient_accumulation_steps == 0:

            # 옵티마이저가 모델의 파라미터를 업데이트
            optimizer.step()

            # gradient와 옵티마이저가 메모리에서 차지하는 용량 추정
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)

            # 메모리 사용량 출력
            if not gpu_utilization_printed:
                print_gpu_utilization("train_model")
                gpu_utilization_printed = True

            # 옵티마이저의 gradient 초기화; 안하면 계속 축적되어 잘못된 업데이트가 발생함.
            optimizer.zero_grad()

    print(f"옵티마이저 상태의 메모리 사용량: {optimizer_memory / (1024 ** 3):.3f} GB")
    print(f"그레디언트 메모리 사용량: {gradients_memory / (1024 ** 3):.3f} GB")

# 랜덤 데이터셋을 생성하는 make_dummy_dataset 정의

In [ ]:
import numpy as np
from datasets import Dataset

def make_dummy_dataset():
  seq_len, dataset_size = 256, 64
  dummy_data = {
      # 64 X 256 크기의 행렬을 랜덤으로 만드는데, 각 요소는 100부터 30000사이의 무작위 값을 가진다.
      "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)), # 토큰 아이디
      "labels": np.random.randint(100, 30000, (dataset_size, seq_len)), # 라벨
  }
  dataset = Dataset.from_dict(dummy_data) # dummy data 딕셔너리를 Dataseet 객체로 변환함.
  dataset.set_format("pt") # Pytorch의 텐서 형식으로 데이터셋의 형식을 설정
  return dataset

# 더이상 사용하지 않는 GPU 메모리를 반환하는 cleanup 함수

In [ ]:
import gc # garbage collection

def cleanup():
    if 'model' in globals():
        del globals()['model']
    if 'dataset' in globals():
        del globals()['dataset']
    gc.collect() # 더 이상 사용되지 않는 객체를 찾아서 메모리에서 강제로 해제
    torch.cuda.empty_cache() # GPU 캐시 초기화

# GPU 사용량을 확인하는 gpu_memory_experiment 함수 정의

In [ ]:
from transformers import TrainingArguments, Trainer

def gpu_memory_experiment(batch_size,
                          gradient_accumulation_steps=1,
                          gradient_checkpointing=False,
                          model_id="EleutherAI/polyglot-ko-1.3b",
                          peft=None):

    print(f"배치 사이즈: {batch_size}")

    model, tokenizer = load_model_and_tokenizer(model_id, peft=peft)
    if gradient_checkpointing == True or peft == 'qlora':
        model.config.use_cache = False # 모델이 중간 계산 결과를 캐싱하지 않도록 함.

    dataset = make_dummy_dataset() # 더미 데이터셋 생성

    training_args = TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        gradient_checkpointing=gradient_checkpointing,
        output_dir="./result", # 결과 저장
        num_train_epochs=1 # 에폭은 한번
      )

    try:
        train_model(model, dataset, training_args) # 학습을 하긴 할건데
    except RuntimeError as e: # 에러가 발생하면 발생했다로 로그 찍음
        if "CUDA out of memory" in str(e):
            print(e)
        else:
            raise e
    finally: # 다 끝나면 메모리를 해제해주기
        del model, dataset
        gc.collect()
        torch.cuda.empty_cache()
        print_gpu_utilization()

# 배치 사이즈를 변경하며 메모리 사용량 측정

In [ ]:
cleanup()
print_gpu_utilization()

for batch_size in [4, 8, 16]:
    gpu_memory_experiment(batch_size)

    torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[] GPU 메모리 사용량: 2.484 GB
배치 사이즈: 4


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[load_model_and_tokenizer] GPU 메모리 사용량: 2.599 GB


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

[] GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[load_model_and_tokenizer] GPU 메모리 사용량: 2.615 GB


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


[train_model] GPU 메모리 사용량: 12.164 GB
CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 45.06 MiB is free. Process 6716 has 14.70 GiB memory in use. Of the allocated memory 14.51 GiB is allocated by PyTorch, and 55.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
[] GPU 메모리 사용량: 0.016 GB


# 그레디언트 누적을 적용했을 때 메모리 사용량

In [ ]:
cleanup()
print_gpu_utilization() # GPU 사용량을 반환함

gpu_memory_experiment(batch_size=4, gradient_accumulation_steps=4)

torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[] GPU 메모리 사용량: 0.016 GB
배치 사이즈: 4


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[load_model_and_tokenizer] GPU 메모리 사용량: 2.615 GB


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


[train_model] GPU 메모리 사용량: 10.586 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
[] GPU 메모리 사용량: 0.016 GB


# 그레디언트 체크포인팅 사용 시 메모리 사용량

In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, gradient_checkpointing=True) # True로 설정해주는 것만으로 알아서 체크포인팅이 가능하다

torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[] GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[load_model_and_tokenizer] GPU 메모리 사용량: 2.615 GB


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


[train_model] GPU 메모리 사용량: 10.290 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
[] GPU 메모리 사용량: 0.016 GB


# 모델을 불러오면서 LoRA 적용하기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    elif peft == 'lora': # 이 부분이 기존의 코드에서 추가되었다
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        lora_config = LoraConfig(
                    r=8, # d x d = (d x r) x (r x d); r이 너무 작으면 정밀도가 떨어짐; 적당한 값은 시도를 통해 구할 수 있음
                    lora_alpha=32, # 새롭게 만든 A, B 행렬의 중요도(학습의 영향)를 결정한다; alpha/r 만큼 A,B 가 만드는 업데이트의 영향이 올라감
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )

        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    print_gpu_utilization()
    return model, tokenizer

# LoRA를 적용했을 때 GPU 메모리 사용량 확인

In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, peft='lora')

torch.cuda.empty_cache()

[] GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.11796039111242178
[] GPU 메모리 사용량: 2.618 GB


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


[train_model] GPU 메모리 사용량: 4.732 GB
옵티마이저 상태의 메모리 사용량: 0.006 GB
그레디언트 메모리 사용량: 0.003 GB
[] GPU 메모리 사용량: 0.016 GB


# 4비트 양자화 모델 불러오기

In [ ]:
from transformers import BitsAndBytesConfig
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True, # 4비트 양자화를 하겠다
    bnb_4bit_quant_type="nf4", # 4비트의 형식은 nf4를 쓰겠다
    bnb_4bit_use_double_quant=True, # 2차 양자화를 하겠다
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# QLoRA 모델을 불러오는 부분을 추가

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    elif peft == 'lora':
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )

        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
    elif peft == 'qlora': # 위 함수에서 여기 elif 문만 추가 시켜 준 것임. 나머진 동일
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )
        bnb_config = BitsAndBytesConfig(
                  load_in_4bit=True,
                  bnb_4bit_use_double_quant=True,
                  bnb_4bit_quant_type="nf4",
                  bnb_4bit_compute_dtype=torch.float16
              )
        model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model)
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    print_gpu_utilization()
    return model, tokenizer

# QLoRA를 적용했을 때 GPU 메모리 사용량 확인

In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, peft='qlora')

torch.cuda.empty_cache()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[] GPU 메모리 사용량: 0.945 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.11796039111242178
[] GPU 메모리 사용량: 2.112 GB


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad()

[train_model] GPU 메모리 사용량: 2.651 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레디언트 메모리 사용량: 0.006 GB
[] GPU 메모리 사용량: 0.945 GB
